In [1]:
import pandas as pd 
import numpy as np 
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval

In [2]:
df1=pd.read_csv('data/tmdb_5000_credits.csv')
df2=pd.read_csv('data/tmdb_5000_movies.csv')

In [3]:
df = pd.merge(
    df2, df1.drop(['title'], axis=1),
    how='left', left_on = 'id', right_on = 'movie_id'
).drop(['movie_id' ], axis=1)

In [4]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


##### A recomendação *Content Based*, nos fornece uma recomendação por similiaridade, ou seja, dado um item o modelo retorna items similares. Neste caso trabalharemos com duas perspectivas para definir similaridade:
1. Baseado na Descrição (conteúdo: Movie Overviews
2. Baseado em Metadados (conteúdo : Movie Cast, Crew, Keywords e Genre)

A técnica utilizada para definir similaridade é a *Similariade dos Cossenos*, definida por:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $


#### Content Based - Descrição do Filme

##### O primeiro passo é extrair do campo overview as palavras que realmente descrevem o filme. Isto será feito em dois passos:
1. Extração das stop words, que são as palavras vazias. Em geral elas são artigos, preposições e/ou as palavras mais comuns de uma determinada língua.
2. Aplicação da técnica *Term Frequency-Inverse Document Frequency (TF-IDF)*. Esta técnica é uma medida estatística que indica a importância de uma palavra de um documento em relação a uma coleção de documentos, baseada na sua frequência inversa.
3. Com a importância de cada palavra em mãos, podemos então calcular a similaridade. Neste caso, usaremos a similaridade de cossenos 

In [5]:
tfidf = TfidfVectorizer(stop_words='english')

In [6]:
df['overview'] = df['overview'].fillna('')

In [7]:
tfidf_matrix = tfidf.fit_transform(df['overview'])

In [8]:
tfidf_matrix.shape

(4803, 20978)

In [9]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
cosine_sim

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.02160533,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        , ...,  0.01488159,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.02160533,  0.01488159, ...,  1.        ,
         0.01609091,  0.00701914],
       [ 0.        ,  0.        ,  0.        , ...,  0.01609091,
         1.        ,  0.01171696],
       [ 0.        ,  0.        ,  0.        , ...,  0.00701914,
         0.01171696,  1.        ]])

##### cosine_sim retorna uma matriz indicando a similaridade entre os filmes

In [11]:
# Precisamos associar os índices aos nomes dos filmes, uma vez que a recomendação será baseada no nome dos filmes.
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [12]:
def recomendacao(title, df, indices, col, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df[col].iloc[movie_indices]

In [13]:
recomendacao('Avatar', df, indices, 'title')

3604                       Apollo 18
2130                    The American
634                       The Matrix
1341            The Inhabited Island
529                 Tears of the Sun
1610                           Hanna
311     The Adventures of Pluto Nash
847                         Semi-Pro
775                        Supernova
2628             Blood and Chocolate
Name: title, dtype: object

In [14]:
recomendacao('The Godfather', df, indices, 'title')

2731     The Godfather: Part II
1873                 Blood Ties
867     The Godfather: Part III
3727                 Easy Money
3623                       Made
3125                     Eulogy
3896                   Sinister
4506            The Maid's Room
3783                        Joe
2244      The Cold Light of Day
Name: title, dtype: object

#### Content Based - Metadados do Filme

Diferente da etapa anterior, substiuiremos a descrição de cada filme, por uma "descrição" baseado em alguns metadados. Esses metadados nada mais são do ques as informações de elenco, keywords, gênero e diretores agregadas.

In [15]:
def filt_values(dataset, col, n, key):
    df = dataset[col].fillna('[]').apply(ast.literal_eval)
    df = df.apply(lambda x: [i[key] for i in x] if isinstance(x, list) else []).to_frame()
    df[col] = df.apply(lambda x: [df[col][x][:n] for x in range(len(df))])
    return df

In [16]:
def get_director(dataset, col):
    df = dataset[col].fillna('[]').apply(ast.literal_eval)
    df = df.apply(lambda x: [i['name'] for i in x if i['job'] == 'Director'] if isinstance(x, list)  else []).to_frame()
    return df

In [17]:
df_item_based = pd.DataFrame()

In [18]:
df_item_based['cast'] = filt_values(df, 'cast', 3, 'character').apply(lambda x: x.astype(str).str.lower())
df_item_based['keywords'] = filt_values(df, 'keywords', 3, 'name').apply(lambda x: x.astype(str).str.lower())
df_item_based['genres'] = filt_values(df, 'genres', 3, 'name').apply(lambda x: x.astype(str).str.lower())
df_item_based['director'] = get_director(df, 'crew').apply(lambda x: x.astype(str).str.lower())

In [19]:
df_item_based.head()

,cast,keywords,genres,director
0,"['jake sully', 'neytiri', 'dr. grace augustine']","['culture clash', 'future', 'space war']","['action', 'adventure', 'fantasy']",['james cameron']
1,"['captain jack sparrow', 'will turner', 'eliza...","['ocean', 'drug abuse', 'exotic island']","['adventure', 'fantasy', 'action']",['gore verbinski']
2,"['james bond', 'blofeld', 'madeleine']","['spy', 'based on novel', 'secret agent']","['action', 'adventure', 'crime']",['sam mendes']
3,"['bruce wayne / batman', 'alfred pennyworth', ...","['dc comics', 'crime fighter', 'terrorist']","['action', 'crime', 'drama']",['christopher nolan']
4,"['john carter', 'dejah thoris', 'sola']","['based on novel', 'mars', 'medallion']","['action', 'adventure', 'science fiction']",['andrew stanton']


Com a base pronta, aplicaremos os seguintes passos:

1. CountVectorizer - retorna uma matriz com a contagem dos tokens de cada filme. Diferente do passo anterior, aqui não precisamos analisar a frequência inversa uma vez que jaá extraimos a informação necessária (O conteúdo foi transforma para lowercase para padronização).
2. Similaridade dos cossenos

In [20]:
df_item_based['soup'] = df_item_based['keywords'] + df_item_based['cast'] + df_item_based['director'] + df_item_based['genres']
df_item_based['soup'] = df_item_based['soup'].apply(lambda x: ''.join(x)) 

In [21]:
df_item_based.head()

,cast,keywords,genres,director,soup
0,"['jake sully', 'neytiri', 'dr. grace augustine']","['culture clash', 'future', 'space war']","['action', 'adventure', 'fantasy']",['james cameron'],"['culture clash', 'future', 'space war']['jake..."
1,"['captain jack sparrow', 'will turner', 'eliza...","['ocean', 'drug abuse', 'exotic island']","['adventure', 'fantasy', 'action']",['gore verbinski'],"['ocean', 'drug abuse', 'exotic island']['capt..."
2,"['james bond', 'blofeld', 'madeleine']","['spy', 'based on novel', 'secret agent']","['action', 'adventure', 'crime']",['sam mendes'],"['spy', 'based on novel', 'secret agent']['jam..."
3,"['bruce wayne / batman', 'alfred pennyworth', ...","['dc comics', 'crime fighter', 'terrorist']","['action', 'crime', 'drama']",['christopher nolan'],"['dc comics', 'crime fighter', 'terrorist']['b..."
4,"['john carter', 'dejah thoris', 'sola']","['based on novel', 'mars', 'medallion']","['action', 'adventure', 'science fiction']",['andrew stanton'],"['based on novel', 'mars', 'medallion']['john ..."


In [22]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df_item_based['soup'])

In [23]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [24]:
cosine_sim2

array([[ 1.        ,  0.12903226,  0.13826023, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.12903226,  1.        ,  0.06913011, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13826023,  0.06913011,  1.        , ...,  0.        ,
         0.06415003,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.06415003, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [25]:
df_item_based['title'] = df['title']

In [26]:
df_item_based = df_item_based.reset_index()
indices = pd.Series(df_item_based.index, index=df_item_based['title'])

In [27]:
recomendacao('The Dark Knight Rises',df_item_based, indices, 'title', cosine_sim=cosine_sim2)

119                               Batman Begins
65                              The Dark Knight
4638                   Amidst the Devil's Wings
299                              Batman Forever
210                              Batman & Robin
3854    Batman: The Dark Knight Returns, Part 2
3966                                Point Blank
1359                                     Batman
2793                       The Killer Inside Me
9            Batman v Superman: Dawn of Justice
Name: title, dtype: object